In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2873566a38e53e9d55b89faec945e704d2a1136e57ff686b152dbcfb1a1fe68b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
!pip install graphframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 4.2 MB/s 


In [ ]:
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file 
100   166  100   166    0     0    468      0 --:--:-- --:--:-- --:--:--   468
curl: (23) Failed writing body (0 != 166)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 123941 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u342-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u342-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u342-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u342-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u342-b07-0ubuntu1~18.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting up ope

In [ ]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

In [ ]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()

In [ ]:
from pyspark.sql.types import *
from graphframes import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def create_transport_graph():
    node_fields = [
        StructField("id", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    nodes = spark.read.csv("transport-nodes.csv", header=True,
                           schema=StructType(node_fields))

    rels = spark.read.csv("transport-relationships.csv", header=True)
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst")
                     .select("src", "dst", "relationship", "cost"))
    
    relationships = rels.union(reversed_rels)

    return GraphFrame(nodes, relationships)

In [ ]:
g = create_transport_graph()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:149: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  "DataFrame.sql_ctx is an internal property, and will be removed "


In [ ]:
(g.vertices
 .filter("population > 100000 and population < 300000")
 .sort("population")
 .show())

+----------+--------+---------+----------+
|        id|latitude|longitude|population|
+----------+--------+---------+----------+
|Colchester|51.88921|  0.90421|    104390|
|   Ipswich|52.05917|  1.15545|    133384|
+----------+--------+---------+----------+



In [ ]:
from_expr = "id = 'Den Haag'"
to_expr = "population > 100000 and population < 300000 and id <> 'Den Haag'"
result = g.bfs(from_expr,to_expr)
print(result.columns)

['from', 'e0', 'v1', 'e1', 'v2', 'e2', 'to']


/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
columns = [columns for columns in result.columns if not columns.startswith("e")]
result.select(columns).show()

+--------------------+--------------------+--------------------+--------------------+
|                from|                  v1|                  v2|                  to|
+--------------------+--------------------+--------------------+--------------------+
|{Den Haag, 52.078...|{Hoek van Holland...|{Felixstowe, 51.9...|{Ipswich, 52.0591...|
+--------------------+--------------------+--------------------+--------------------+



In [ ]:
import time
import numpy as np
from random import random
from DisjointSet import DisjointSet

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
data_set = "netsience.net"

In [ ]:
def get_nodes_edges(file):
    nodes, edges = set(), list()
    cnt = 0
    with open(file, "r") as f:
        for line in f.readlines():
            line = line.strip().split()
            nodes.add(line[0])
            nodes.add(line[1])
            edges.append((line[0], line[1], line[2] if len(line) is 3 else 1.0))
        
    return nodes, edges, len(nodes), len(edges)

def save_result(file, lists):
    lists = [str(line) + "\n" for line in lists]
    with open(file, "w") as f:
        f.writelines(lists)

In [ ]:
# 데이터 셋을 노드와 에지 데이터로 전환
print("Convert file to nodes and edges %s" % data_set)
t0 = time.time()
nodes, edges, num_nodes, num_edges = get_nodes_edges(data_set)
t1 = time.time()
print("Convert finish %s (s)" % str(t1 - t0))
print("%s nodes and %s edges" % (str(num_nodes), str(num_edges)))

Convert file to nodes and edges netsience.net
Convert finish 0.006144046783447266 (s)
1461 nodes and 2742 edges


In [ ]:
def kruskal(nodes, edges):
    # 최소신장트리 에지
    mst = []

    # 노드 초기화
    forest = DisjointSet(nodes)
    
    # 가중치로 에지 정렬
    edges = sorted(edges, key=lambda x: x[2])
    # 최소신장트리 최대 에지 수 == 노드 수
    num_edges = len(nodes)

    # kruskal's algorithm
    for (src, dst, weight) in edges:
        # 전체 노드set에서 노드가 없을 경우 skip
        if src not in nodes or dst not in nodes:
            continue
        # 각각 시작 노드와 도착 노드의 부모를 찾는다.
        if forest.union(src, dst):
            # 순환을 만들지 않는 경우 최소신장트리에 현재 에지를 추가합니다.
            mst.append((src, dst, weight))
            # 조기 종료
            if len(mst) == num_edges:
                break

    return mst

In [ ]:
def vertex_partitioning(sc, nodes, edges, num_partition=4):
    # 서로 다른 정점 파티션의 조합을 계산하는 함수 정의
    def combine(x):
        if x[0] < x[1]:
          return x[0] + x[1]
        
    vertices = sc.parallelize(nodes, num_partition).glom()
    vertices = vertices.cartesian(vertices).map(combine).filter(lambda x: x is not None)
   
    # parallelize
    vertices = sc.parallelize(vertices.collect(), num_partition)

    # local 최소신장트리를 계산하는 함수
    def local_kruskal(iterator):
        for subset in iterator:
            for edge in kruskal(nodes=set(subset), edges=edges):
                yield edge
    # local 최소신장트리를 계산
    subtrees = vertices.mapPartitions(local_kruskal).distinct().collect()

    # global 최소신장트리를 계산
    return kruskal(nodes=nodes, edges=subtrees)

In [ ]:
# 정점 partitioning 후 크루스칼 알고리즘 실행
t2 = time.time()
vertex_partitioning_result = vertex_partitioning(sc, nodes, edges, num_partition=4)
t3 = time.time()
print("Vertex_partitioning %s (s)" % str(t2 - t3))
print("%s edges" % len(vertex_partitioning_result))
# 결과 값 저장
save_result("vertex_partitioning_result.txt", vertex_partitioning_result)

Vertex_partitioning -2.987013339996338 (s)
1193 edges


In [ ]:
# Kruskal
# t4 = time.time()
# kruskal_result = kruskal(nodes, edges)
# t5 = time.time()
# print("Kruskal %s (s)" % str(t5 - t4))
# print("%s edges" % len(kruskal_result))
# save_result("kruskal_result.txt", kruskal_result)

Kruskal 0.013344287872314453 (s)
1193 edges


In [ ]:
from graphframes.lib import AggregateMessages as AM

In [ ]:
add_path_udf = F.udf(lambda path, id : path + [id], ArrayType(StringType()))

In [ ]:
def shortest_path(g, origin, destination, column_name="cost"):
    if g.vertices.filter(g.vertices.id == destination).count() == 0:
        return (spark.createDataFrame(sc.emptyRDD(), g.vertices.schema)
                .withColumn("path", F.array()))

    vertices = (g.vertices.withColumn("visited", F.lit(False))
                .withColumn("distance", F.when(g.vertices["id"] == origin, 0)
                            .otherwise(float("inf")))
                .withColumn("path", F.array()))
    cached_vertices = AM.getCachedDataFrame(vertices)
    g2 = GraphFrame(cached_vertices, g.edges)

    while g2.vertices.filter('visited == False').first():
        current_node_id = g2.vertices.filter('visited == False').sort("distance").first().id

        msg_distance = AM.edge[column_name] + AM.src['distance']
        msg_path = add_path_udf(AM.src["path"], AM.src["id"])
        msg_for_dst = F.when(AM.src['id'] == current_node_id, F.struct(msg_distance, msg_path))
        new_distances = g2.aggregateMessages(F.min(AM.msg).alias("aggMess"),
                                             sendToDst=msg_for_dst)

        new_visited_col = F.when(
            g2.vertices.visited | (g2.vertices.id == current_node_id), True).otherwise(False)
        new_distance_col = F.when(new_distances["aggMess"].isNotNull() &
                                  (new_distances.aggMess["col1"] < g2.vertices.distance),
                                  new_distances.aggMess["col1"]) \
            .otherwise(g2.vertices.distance)
        new_path_col = F.when(new_distances["aggMess"].isNotNull() &
                              (new_distances.aggMess["col1"] < g2.vertices.distance),
                              new_distances.aggMess["col2"].cast("array<string>")) \
            .otherwise(g2.vertices.path)

        new_vertices = (g2.vertices.join(new_distances, on="id", how="left_outer")
                        .drop(new_distances["id"])
                        .withColumn("visited", new_visited_col)
                        .withColumn("newDistance", new_distance_col)
                        .withColumn("newPath", new_path_col)
                        .drop("aggMess", "distance", "path")
                        .withColumnRenamed('newDistance', 'distance')
                        .withColumnRenamed('newPath', 'path'))
        cached_new_vertices = AM.getCachedDataFrame(new_vertices)
        g2 = GraphFrame(cached_new_vertices, g2.edges)
        if g2.vertices.filter(g2.vertices.id == destination).first().visited:
            return (g2.vertices.filter(g2.vertices.id == destination)
                    .withColumn("newPath", add_path_udf("path", "id"))
                    .drop("visited", "path")
                    .withColumnRenamed("newPath", "path"))
    return (spark.createDataFrame(sc.emptyRDD(), g.vertices.schema)
            .withColumn("path", F.array()))

In [ ]:
result = shortest_path(g, "Amsterdam","Colchester","cost")
result.select("id","distance","path").show(truncate=False)

+----------+--------+------------------------------------------------------------------------+
|id        |distance|path                                                                    |
+----------+--------+------------------------------------------------------------------------+
|Colchester|347.0   |[Amsterdam, Den Haag, Hoek van Holland, Felixstowe, Ipswich, Colchester]|
+----------+--------+------------------------------------------------------------------------+



In [ ]:
result = shortest_path(g, "Amsterdam","Colchester","cost")
result.select("id","distance","path").show(truncate=False)

+----------+--------+------------------------------------------------------------------------+
|id        |distance|path                                                                    |
+----------+--------+------------------------------------------------------------------------+
|Colchester|347.0   |[Amsterdam, Den Haag, Hoek van Holland, Felixstowe, Ipswich, Colchester]|
+----------+--------+------------------------------------------------------------------------+



In [ ]:
add_path_udf = F.udf(lambda path, id: path + [id], ArrayType(StringType()))

In [ ]:
def sssp(g, origin, column_name="cost"):
    vertices = g.vertices \
        .withColumn("visited", F.lit(False)) \
        .withColumn("distance",
            F.when(g.vertices["id"] == origin, 0).otherwise(float("inf"))) \
        .withColumn("path", F.array())
    cached_vertices = AM.getCachedDataFrame(vertices)
    g2 = GraphFrame(cached_vertices, g.edges)

    while g2.vertices.filter('visited == False').first():
        current_node_id = g2.vertices.filter('visited == False').sort("distance").first().id

        msg_distance = AM.edge[column_name] + AM.src['distance']
        msg_path = add_path_udf(AM.src["path"], AM.src["id"])
        msg_for_dst = F.when(AM.src['id'] == current_node_id, F.struct(msg_distance, msg_path))
        new_distances = g2.aggregateMessages(
            F.min(AM.msg).alias("aggMess"), sendToDst=msg_for_dst)

        new_visited_col = F.when(
            g2.vertices.visited | (g2.vertices.id == current_node_id), True).otherwise(False)
        new_distance_col = F.when(new_distances["aggMess"].isNotNull() &
                                  (new_distances.aggMess["col1"] < g2.vertices.distance),
                                  new_distances.aggMess["col1"]) \
                            .otherwise(g2.vertices.distance)
        new_path_col = F.when(new_distances["aggMess"].isNotNull() &
                              (new_distances.aggMess["col1"] < g2.vertices.distance),
                              new_distances.aggMess["col2"].cast("array<string>")) \
                        .otherwise(g2.vertices.path)

        new_vertices = g2.vertices.join(new_distances, on="id", how="left_outer") \
            .drop(new_distances["id"]) \
            .withColumn("visited", new_visited_col) \
            .withColumn("newDistance", new_distance_col) \
            .withColumn("newPath", new_path_col) \
            .drop("aggMess", "distance", "path") \
            .withColumnRenamed('newDistance', 'distance') \
            .withColumnRenamed('newPath', 'path')
        cached_new_vertices = AM.getCachedDataFrame(new_vertices)
        g2 = GraphFrame(cached_new_vertices, g2.edges)

    return g2.vertices \
                .withColumn("newPath", add_path_udf("path", "id")) \
                .drop("visited", "path") \
                .withColumnRenamed("newPath", "path")


In [ ]:
via_udf = F.udf(lambda path: path[1:-1], ArrayType(StringType()))

result = sssp(g, "Amsterdam", "cost")
(result
 .withColumn("via", via_udf("path"))
 .select("id", "distance", "via")
 .sort("distance")
 .show(truncate=False))

+----------------+--------+-------------------------------------------------------------+
|id              |distance|via                                                          |
+----------------+--------+-------------------------------------------------------------+
|Amsterdam       |0.0     |[]                                                           |
|Utrecht         |46.0    |[]                                                           |
|Den Haag        |59.0    |[]                                                           |
|Gouda           |81.0    |[Utrecht]                                                    |
|Rotterdam       |85.0    |[Den Haag]                                                   |
|Hoek van Holland|86.0    |[Den Haag]                                                   |
|Felixstowe      |293.0   |[Den Haag, Hoek van Holland]                                 |
|Ipswich         |315.0   |[Den Haag, Hoek van Holland, Felixstowe]                     |
|Colcheste